# Lab 2 - Analyser un Appel d'Offre avec l'IA

## Cas d'usage

Dans le monde de l'avant-vente, la réactivité et la pertinence sont des facteurs clés de succès. Répondre à un appel d'offre (RFP - Request for Proposal) est un processus chronophage qui demande de bien cerner les besoins du client pour proposer une solution adaptée. 

Ce laboratoire démontre comment une IA agentique simple peut accélérer drastiquement ce processus. Nous allons utiliser **LangChain**, un framework puissant qui agit comme un "chef d'orchestre" pour les grands modèles de langage (LLMs), afin de :

1.  Lire et comprendre un appel d'offre.
2.  En extraire les informations stratégiques.
3.  Générer une première ébauche de proposition technique.

### Étape 1 : Charger le document d'appel d'offre

In [ ]:
from langchain.document_loaders import TextLoader

loader = TextLoader('./appel_offre.txt')
document = loader.load()

print(document[0].page_content)

### Étape 2 : Créer une chaîne d'extraction d'informations

Nous allons définir un `PromptTemplate` qui guidera le LLM pour qu'il identifie précisément les points qui nous intéressent.

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI # Assurez-vous d'avoir configuré votre clé API OpenAI

# Initialiser le modèle
# Remplacez par votre modèle de prédilection si nécessaire (ex: via Ollama, Mistral, etc.)
# Pensez à configurer votre clé API, par exemple avec : 
# import os
# os.environ['OPENAI_API_KEY'] = 'VOTRE_CLE_ICI'
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Créer le template de prompt pour l'extraction
extraction_template = """
Lis attentivement le texte de l'appel d'offre suivant et extrais les informations clés dans un format JSON valide.

Texte de l'appel d'offre:
--- 
{document_text}
--- 

Extrais les informations suivantes:
1. 'objectif_metier': L'objectif principal que le client cherche à atteindre.
2. 'exigences_techniques': Les contraintes ou technologies spécifiques demandées.
3. 'date_limite': La date ou période de livraison attendue.

Ne retourne que le JSON, sans aucun autre commentaire ou texte d'introduction.
"""

extraction_prompt = PromptTemplate(
    input_variables=["document_text"],
    template=extraction_template
)

extraction_chain = LLMChain(llm=llm, prompt=extraction_prompt)

print("Chaîne d'extraction créée.")

### Étape 3 : Exécuter la chaîne et extraire les points clés

In [ ]:
import json

raw_result = extraction_chain.run(document_text=document[0].page_content)
extracted_data = json.loads(raw_result)

print("Informations extraites de l'appel d'offre :\n")
print(json.dumps(extracted_data, indent=2, ensure_ascii=False))

### Étape 4 : Créer une chaîne de génération de proposition

In [ ]:
# Créer le template de prompt pour la génération
generation_template = """
Agis en tant qu'architecte de solutions IA. En te basant sur les informations extraites de l'appel d'offre, rédige une ébauche de proposition technique en 3 points clairs et concis.

Informations extraites:
--- 
Objectif métier du client: {objectif_metier}
Exigences techniques: {exigences_techniques}
Date limite: {date_limite}
--- 

Structure ta proposition comme suit:
1. **Approche proposée:** Décris brièvement la solution envisagée pour atteindre l'objectif métier.
2. **Technologies clés:** Liste les technologies qui seront utilisées, en accord avec les exigences.
3. **Livrables:** Précise ce qui sera concrètement livré au client à la date limite.
"""

generation_prompt = PromptTemplate(
    input_variables=["objectif_metier", "exigences_techniques", "date_limite"],
    template=generation_template
)

generation_chain = LLMChain(llm=llm, prompt=generation_prompt)

print("Chaîne de génération créée.")

In [ ]:
# Exécuter la chaîne de génération
proposition = generation_chain.run(extracted_data)

print("--- ÉBAUCHE DE PROPOSITION TECHNIQUE ---\n")
print(proposition)

## Conclusion

En quelques minutes, nous avons automatisé une partie significative du travail d'analyse d'un appel d'offre et de rédaction d'une proposition. Les gains sont multiples :

*   **Gain de temps :** L'analyse et la première ébauche sont quasi-instantanées.
*   **Standardisation :** La structure des réponses est homogène, ce qui facilite la relecture et la validation.
*   **Fiabilité :** L'IA est moins susceptible d'oublier une information clé lors de la lecture du document.

Dans un vrai projet, cet agent pourrait être enrichi avec des **outils** (`tools`) lui donnant accès à :

*   Une recherche web pour se renseigner sur l'entreprise cliente.
*   Une base de connaissances interne (via RAG) pour réutiliser des briques de projets précédents.
*   Un outil de pricing pour estimer le coût du projet.